In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.integrate as integrate
import ipyvolume as ipv
from importlib import reload

import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

In [ ]:
importlib.reload(ipv)

In [ ]:
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111)

def noise(x):
    return np.sin(x)

def fbm(octaves, lacunarity, gain, x):
    # initial values
    amplitude = 0.5;
    frequency = 1.;

    #  loop of octaves
    y = np.zeros_like(x)
    for i in range(int(octaves)):
        y += amplitude * noise(frequency*x);
        frequency *= lacunarity;
        amplitude *= gain;
    return y

def plot_fbm(octaves, lacunarity, gain):
    x = np.arange(-10, 10, 0.1)
    y = fbm(octaves, lacunarity, gain, x)
    ax.clear()
    ax.plot(x, y)

widgets.interact(plot_fbm, octaves=(1, 10, 1.0), lacunarity=(0, 10.0, 0.1), gain=(0, 2, 0.1))

In [ ]:
import sampling as sampling
import spherical as sph
import BRDF as BRDF
import LTC as LTC

import mitsuba.python.chi2 as chi2

import importlib
importlib.reload(sampling)
importlib.reload(sph)
importlib.reload(BRDF)
importlib.reload(LTC)

# samples_ = np.random.uniform(size =[2,1024*32])
# ggx = sampling.NormalDistributionGGX()
# alpha = 0.5
# Wh = ggx.sample(alpha, samples_)
# pdf = ggx.pdf(alpha, Wh)
# D = ggx.eval(alpha, Wh)


ltc = LTC.LTC(0.2, 0.2, 1)

def my_sample(sample):
    return ltc.sample(sample)
    return ggx.sample(alpha, sample)
    return square_to_cosine_hemisphere(sample)

def my_pdf(p):
    return ltc.pdf(p)
    return ggx.pdf(alpha, p)
    return square_to_cosine_hemisphere_pdf(p)

chi2Test = chi2.ChiSquareTest(
    domain=chi2.SphericalDomain(),
    sample_func=my_sample,
    pdf_func=my_pdf,
    sample_dim=2,
    sample_count=1000000,
    ires=32
)

chi2Test.tabulate_histogram()
chi2Test.tabulate_pdf(simpson_rule=True)
chi2Test.run()

fig, ax = plt.subplots(2)
fig.set_figwidth(8)
fig.set_figheight(8)
vmax = np.maximum(np.amax(chi2Test.histogram), np.amax(chi2Test.pdf))
# vmax *= 0.96
ax[0].set_title('Observed frequencies')
ax[0].imshow(chi2Test.histogram, cmap='jet', vmax=vmax)
ax[1].set_title('Expected frequencies')
ax[1].imshow(chi2Test.pdf, cmap='jet', vmax=vmax)
# print(chi2Test.histogram.shape)

# vmax = np.amax(chi2Test.histogram)
# vmax = np.maximum(vmax, np.amax(chi2Test.pdf))
# plt.imshow(chi2Test.histogram, cmap='jet', vmax=vmax)
# print(chi2Test.histogram.shape)
# # print(chi2Test.histogram.shape)
# # chi2.run()
# # chi2._dump_tables()

In [ ]:
print(vmax)
print(np.amax(chi2Test.histogram))
print(np.amax(chi2Test.pdf))

In [ ]:

import LTC as LTC
import spherical as sph
importlib.reload(LTC)
importlib.reload(sph)

ltc = LTC.LTC()
theta, phi = sph.meshgrid_spherical_coord(128)
Wi = sph.spherical_dir(theta, phi)
D = ltc.eval(Wi) * np.pi

In [ ]:
import os
import numpy as np
import mitsuba

# Set the desired mitsuba variant
mitsuba.set_variant('scalar_rgb')

from mitsuba.core import Bitmap, Struct, Thread
from mitsuba.core.xml import load_file

# Absolute or relative path to the XML file
filename = 'W:/Mitsuba/mitsuba2/resources/data/scenes/cbox/cbox.xml'

# Add the scene directory to the FileResolver's search path
Thread.thread().file_resolver().append(os.path.dirname(filename))

# Load the actual scene
scene = load_file(filename)

# Call the scene's integrator to render the loaded scene
scene.integrator().render(scene, scene.sensors()[0])

# After rendering, the rendered data is stored in the film
film = scene.sensors()[0].film()

# Write out rendering as high dynamic range OpenEXR file
film.set_destination_file('W:/Mitsuba/mitsuba2/resources/data/scenes/cbox/output.exr')
film.develop()

# Write out a tonemapped JPG of the same rendering
bmp = film.bitmap(raw=True)
bmp.convert(Bitmap.PixelFormat.RGB, Struct.Type.UInt8, srgb_gamma=True).write('W:/Mitsuba/mitsuba2/resources/data/scenes/cbox/output.jpg')

# Get linear pixel values as a numpy array for further processing
bmp_linear_rgb = bmp.convert(Bitmap.PixelFormat.RGB, Struct.Type.Float32, srgb_gamma=False)
image_np = np.array(bmp_linear_rgb)
print(image_np.shape)


In [ ]:
import numpy as np
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_rgb')
from mitsuba.core import Float

import BRDF
import spherical as sph

reload(BRDF)
reload(sph)

import ipyvolume as ipv

alpha = ek.linspace(Float, BRDF.MIN_GGX_ALPHA, 1, 10)
theta = ek.linspace(Float, 0, ek.pi/2, 10)
err0 = np.zeros((10, 10))
err1 = np.zeros((10, 10))

for i in range(len(alpha)):
    for j in range(len(theta)):
        Wo = sph.spherical_dir(theta[j], 0)
#         err0[i][j] = BRDF.maskingFuncProjectedAreaTest(BRDF.NDF_GGX_, BRDF.smithMaskingFunction, alpha[i], Wo, num_samples=1024)[0]
        err1[i][j] = BRDF.whiteFurnaceTest(BRDF.NDF_GGX_, BRDF.smithMaskingFunction, alpha[i], Wo, num_samples=4096)[0]

alpha = alpha.numpy()
theta = theta.numpy()
alpha, theta = np.meshgrid(alpha, theta)
ipv.clear()
ipv.plot_mesh(alpha, theta, err1)
ipv.show()



In [ ]:
import scipy

def func(angle):
    theta = angle * np.pi/2
    return 1 - np.cos(theta)

def func_approx(angle, a, b):
    return angle**2

x = np.arange(0, 1, 0.05)
y = func(x)


# opt, pcov = scipy.optimize.curve_fit(func_approx, x, y)
# y2 = func_approx(x, opt[0], opt[1])
# print(opt)
# print(pcov)
y2 = func_approx(x, 0, 0)
plt.plot(x, y, label="analytic")
plt.plot(x, y2, label="approx")
plt.legend()
plt.show()

In [ ]:
import mitsuba
mitsuba.set_variant("packet_rgb")

import enoki as ek

import spherical as sph
from mitsuba_ext import Frame


def ray_cone_intersection(ray, cone_theta, cone_angle):
    cone_dir = sph.spherical_dir(cone_theta, 0)

    cos_theta = ek.dot(ray, cone_dir)
    return cos_theta > ek.cos(cone_angle)


def cone_integral_analytic(cone_theta, cone_angle):
    return 1 - ek.cos(cone_angle)


def cone_cosine_integral_analytic(cone_theta, cone_angle):
    return ek.cos(cone_theta) * ek.sin(cone_angle)**2


def integral(w, cone_theta, cone_angle, cos_weighted):
    if cos_weighted:
        return ek.select(ray_cone_intersection(w, cone_theta, cone_angle), Frame.cos_theta(w), 0.0) * ek.rcp(ek.pi)
    else:
        return ek.select(ray_cone_intersection(w, cone_theta, cone_angle), Frame.cos_theta(w)*0+1, 0.0) * ek.rcp(ek.pi*2)


In [ ]:
import ipyvolume as ipv
from mitsuba.core import Float

num_grid = 50
cone_theta = ek.linspace(Float, 0.0, ek.pi/2, num_grid)
cone_angle = ek.linspace(Float, 0.0, ek.pi/2, num_grid)
cone_theta, cone_angle = ek.meshgrid(cone_theta, cone_angle)


occlusion_analytic = cone_cosine_integral_analytic(cone_theta, cone_angle)
occlusion_analytic += (np.clip(cone_theta/np.pi + cone_angle/np.pi - 0.5, 0, 1)*1.4)**2

cone_theta_np = cone_theta.numpy().reshape(num_grid, num_grid)
cone_angle_np = cone_angle.numpy().reshape(num_grid, num_grid)
occlusion_analytic_np = occlusion_analytic.numpy().reshape(num_grid, num_grid)
ipv.clear()
ipv.plot_mesh(cone_theta_np, cone_angle_np, occlusion_analytic_np)
ipv.show()



In [ ]:
import ipyvolume as ipv
from mitsuba.core import Float

num_grid = 50
cone_theta = ek.linspace(Float, 0.0, ek.pi/2, num_grid)
cone_angle = ek.linspace(Float, 0.0, ek.pi/2, num_grid)

occlusion_analytic_horiozn_clipping = np.zeros((num_grid, num_grid))
for i, theta in enumerate(cone_theta):
    for j, angle in enumerate(cone_angle):
        occlusion_analytic_horiozn_clipping[j][i] = sph.spherical_integral(lambda w: integral(w, theta, angle, True), hemisphere=True)

cone_theta_np = cone_theta.numpy()
cone_angle_np = cone_angle.numpy()
cone_theta_np, cone_angle_np = np.meshgrid(cone_theta_np, cone_angle_np)
ipv.clear()
ipv.plot_mesh(cone_theta_np, cone_angle_np, occlusion_analytic_horiozn_clipping)
ipv.show()



In [ ]:

diff = occlusion_analytic_horiozn_clipping-occlusion_analytic_np

# def approx(theta, phi):
#     return (np.clip(theta/np.pi + phi/np.pi - 0.5, 0, 1)*1.4)**2

ipv.clear()
ipv.plot_mesh(cone_theta_np, cone_angle_np, diff)
# ipv.plot_mesh(cone_theta_np, cone_angle_np, approx(cone_theta_np, cone_angle_np), color=[0, 1, 0, 1])
ipv.show()



In [ ]:
import mitsuba
mitsuba.set_variant("packet_rgb")
from mitsuba.core import Float

import enoki as ek
import numpy as np

rands = Float(np.random.rand(10))
result = ek.select(rands>0.5, Float(1.0), 0)
print(type(result))

In [ ]:
import mitsuba
mitsuba.set_variant("packet_rgb")

from mitsuba.render import MicrofacetDistribution, MicrofacetType, reflect

from mitsuba_ext import Frame
import spherical as sph

N = sph.spherical_dir(0, 0)
reflect_dir = 
cone_axis = 
theta = 0.3 * np.pi/2

cone_angle = 0.2
roughness = 0.4
alpha = roughness * roughness


def cone_vis(Wi, cone_angle, beta):
    # z axis aligned
    return 1


ggx = MicrofacetDistribution(MicrofacetType.GGX, alpha)
def integrand(Wi):
    beta = ek.acos(dot(cone_axis, reflect_dir))
    ggx.eval(Wi) * Frame.cos_theta(Wi) * cone_vis(Wi, cone_angle, beta)


In [ ]:
import mitsuba
mitsuba.set_variant("packet_rgb")

import enoki as ek

from mitsuba.core import Float, Vector2f, Vector3f
from mitsuba.render import MicrofacetDistribution, MicrofacetType, reflect
from BRDF import linearRoughnessToAlpha

import numpy as np
import matplotlib.pyplot as plt

import spherical as sph

from LTC import BRDFAdapter
import ipyvolume as ipv


def saturate(v):
    return ek.clamp(v, 0, 1)


def specularOcclusion(alphaV, beta, roughness, thetaO):
    thetaC = thetaO - beta
    B = Vector3f(ek.sin(thetaC), 0, ek.cos(thetaC))
    Nx = Vector3f(0, 0, 1)
    Wr = Vector3f(ek.sin(thetaO), 0, ek.cos(thetaO));

    
    localNormal = Nx
    localTangent = ek.normalize(Wr - Nx * ek.dot(Wr, Nx));
    
    cosThetaO = ek.cos(thetaO);
    sinThetaO = ek.sqrt(1-cosThetaO*cosThetaO);
    V = localNormal * cosThetaO - localTangent * sinThetaO;

    
    brdf = BRDFAdapter(roughness, -thetaO)
    
    numVisSamples = 1024
    sample_ = Vector2f(np.random.rand(numVisSamples, 2))
    Wi, pdf = brdf.sample(sample_)
    
    visible = ek.select(ek.acos(ek.dot(B, Wi)) < alphaV, Float(1.0), 0.0)
    accumValue = ek.select(pdf > 0, visible * brdf.eval(Wi) / pdf, 0);
    accumWeight = ek.select(pdf > 0, brdf.eval(Wi) / pdf, 0);

    accumValue = ek.hsum(accumValue)
    accumWeight = ek.hsum(accumWeight)
    
    if (accumWeight > 0):
        return accumValue / accumWeight
    else:
        return 0;




dim = 32
lut = np.zeros((dim, dim))
alphaV = 0.5 * np.pi/2
beta = 0
roughness_ = np.linspace(0.089, 1, dim)
thetaO_ = np.linspace(0, np.pi/2, dim)
roughness_, thetaO_ = np.meshgrid(roughness_, thetaO_)
for k in range(dim):
    for l in range(dim):
        lut[k][l] = specularOcclusion(alphaV, beta, roughness_[k][l], thetaO_[k][l])

        
ipv.clear()
X = roughness_
Y = thetaO_
ipv.plot_surface(X, Y, lut)
ipv.show()


# dim = 16
# lut = np.zeros((dim*dim, dim*dim))
# for i, alphaV in enumerate(np.linspace(0, np.pi/2, dim)):
#     for j, beta in enumerate(np.linspace(0, np.pi/2, dim)):
#         for k, roughness in enumerate(np.linspace(0.089, 1, dim)):
#             for l, thetaO in enumerate(np.linspace(0, np.pi/2, dim)):
#                 s = i * dim
#                 t = j * dim
#                 lut[s + k][t + l] = specularOcclusion(alphaV, beta, roughness, thetaO)
#
# plt.imshow(lut, cmap="Greys")
# plt.show()

# dim = 32
# lut = np.zeros((dim, dim))
# for i, alphaV in enumerate(np.linspace(0, np.pi / 2, dim)):
#     for k, roughness in enumerate(np.linspace(0.089, 1, dim)):
#         for l, thetaO in enumerate(np.linspace(0, np.pi / 2, dim)):
#             beta = thetaO
#             lut = specularOcclusion(alphaV, beta, roughness, thetaO)
#             plt.imshow(lut, cmap="Greys")
#
# plt.show()


In [ ]:
print(specularOcclusion(alphaV, beta, 1, np.pi/2*0.5))

In [ ]:
print(specularOcclusion(alphaV, beta, 0.1, np.pi/2*0.5))

In [ ]:
print(lut)

In [ ]:
import mitsuba
mitsuba.set_variant("packet_rgb")

import enoki as ek

from mitsuba.core import Float, Vector2f, Vector3f
from mitsuba.render import MicrofacetDistribution, MicrofacetType, reflect
from BRDF import linearRoughnessToAlpha

import numpy as np
import matplotlib.pyplot as plt

import spherical as sph

from LTC import BRDFAdapter


def saturate(v):
    return ek.clamp(v, 0, 1)


def specularOcclusion(alphaV, beta, roughness, thetaO):
    thetaC = thetaO - beta
    B = sph.spherical_dir(thetaC, 0)

    numVisSamples = 512
    sample_ = Vector2f(np.random.rand(numVisSamples, 2))

    # negative thetaO
    brdf = BRDFAdapter(roughness, -thetaO)
    Wi, pdf = brdf.sample(sample_)

    visible = ek.select(ek.acos(ek.dot(B, Wi)) < alphaV, Float(1.0), Float(0.0))
    accumValue = ek.select(pdf > 0, visible * brdf.eval(Wi) / pdf, 0)
    accumWeight = ek.select(pdf > 0, brdf.eval(Wi) / pdf, 0)

    accumValue = ek.hsum(accumValue)
    accumWeight = ek.hsum(accumWeight)

    if accumWeight > 0:
        return accumValue / accumWeight
    else:
        return 0


if __name__ == "__main__":
    _SampleCount = 4096
    _ConeHalfAngle = 0.4

    N = Vector3f(0, 0, 1)
    coneDir = N

    # dim = 32
    # lut = np.zeros((32, 16))
    # for i, a  in enumerate(np.linspace(0, 0.1, 32)):
    #     for j, b in enumerate(np.linspace(1, 2.0, 16)):
    #         lut[i][j] = j
    # plt.imshow(lut.transpose(), cmap="Greys")
    # plt.show()

    dim = 32
    lut = np.zeros((dim*dim, dim*dim))
    # beta_ = np.linspace(0, np.pi, dim)
    thetaC_ = np.linspace(np.pi/2, -np.pi/2, dim)
    alphaV_ = np.linspace(0, np.pi/2, dim)
    thetaO_ = np.linspace(0, np.pi / 2, dim)
    roughness_ = np.linspace(0.089, 1, dim)
    # for l, beta in enumerate(beta_):
    for l, thetaC in enumerate(thetaC_):
        beta = thetaO - thetaC
        for k, alphaV in enumerate(alphaV_):
            for i, thetaO in enumerate(thetaO_):
                for j, roughness in enumerate(roughness_):
                    lut[i + l * dim][j + k * dim] = specularOcclusion(alphaV, beta, roughness, thetaO)

    # This import registers the 3D projection, but is otherwise unused.
    # from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
    # fig = plt.figure()
    # ax = fig.gca(projection='3d')
    # X, Y = np.meshgrid(roughness_, thetaO_)
    # surf = ax.plot_surface(X, Y, lut, cmap="coolwarm", linewidth=0, antialiased=False)
    # plt.show()

    plt.imshow(lut, cmap="gray")
    plt.show()

    # dim = 16
    # lut = np.zeros((dim*dim, dim*dim))
    # for i, alphaV in enumerate(np.linspace(0, np.pi/2, dim)):
    #     for j, beta in enumerate(np.linspace(0, np.pi/2, dim)):
    #         for k, roughness in enumerate(np.linspace(0.089, 1, dim)):
    #             for l, thetaO in enumerate(np.linspace(0, np.pi/2, dim)):
    #                 s = i * dim
    #                 t = j * dim
    #                 lut[s + k][t + l] = specularOcclusion(alphaV, beta, roughness, thetaO)
    #
    # plt.imshow(lut, cmap="Greys")
    # plt.show()

    # dim = 32
    # lut = np.zeros((dim, dim))
    # for i, alphaV in enumerate(np.linspace(0, np.pi / 2, dim)):
    #     for k, roughness in enumerate(np.linspace(0.089, 1, dim)):
    #         for l, thetaO in enumerate(np.linspace(0, np.pi / 2, dim)):
    #             beta = thetaO
    #             lut = specularOcclusion(alphaV, beta, roughness, thetaO)
    #             plt.imshow(lut, cmap="Greys")
    #
    # plt.show()


In [ ]:
print(lut.shape)
plt.figure(figsize=(8*32,8*32))
plt.imshow(lut, cmap="gray")
plt.savefig('fuckfuckfuck.png')

In [ ]:
lut_xxx = np.zeros((dim, dim))
for i, thetaO in enumerate(thetaO_):
    for j, roughness in enumerate(roughness_):
        alphaV = 0.5 * np.pi/2
        beta = 0
        lut_xxx[i][j] = specularOcclusion(alphaV, beta, roughness, thetaO)

plt.imshow(lut_xxx, cmap="gray")
plt.show()